In [ ]:
import cv2
import numpy as np

class VideoSynchronizer:
    def __init__(self, video1_path, video2_path):
        self.cap1 = cv2.VideoCapture(video1_path)
        self.cap2 = cv2.VideoCapture(video2_path)
        self.fps1 = self.cap1.get(cv2.CAP_PROP_FPS)
        self.fps2 = self.cap2.get(cv2.CAP_PROP_FPS)
        self.delay = self._calculate_delay()
        
    def _calculate_delay(self):
        """Определение задержки между видео через SIFT-дескрипторы"""
        sift = cv2.SIFT_create()
        
        # Поиск ключевых кадров
        kp1, des1 = self._get_key_frame(self.cap1, sift)
        kp2, des2 = self._get_key_frame(self.cap2, sift)
        
        # Сопоставление дескрипторов
        bf = cv2.BFMatcher()
        matches = bf.knnMatch(des1, des2, k=2)
        
        # Фильтрация по соотношению расстояний
        good = []
        for m,n in matches:
            if m.distance < 0.75*n.distance:
                good.append(m)
                
        # Расчет смещения в кадрах
        time_diff = len(good)/self.fps1
        return int(time_diff * self.fps2)

    def _get_key_frame(self, cap, detector):
        """Поиск кадра с максимальным числом особенностей"""
        max_features = 0
        best_frame = None
        for _ in range(30):  # Проверка первых 30 кадров
            ret, frame = cap.read()
            if not ret: break
            gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            kp, des = detector.detectAndCompute(gray, None)
            if len(kp) > max_features:
                max_features = len(kp)
                best_frame = (kp, des)
        return best_frame

    def get_sync_frames(self):
        """Генератор синхронизированных кадров"""
        self.cap2.set(cv2.CAP_PROP_POS_FRAMES, self.delay)
        
        while True:
            ret1, frame1 = self.cap1.read()
            ret2, frame2 = self.cap2.read()
            
            if not ret1 or not ret2:
                break
                
            yield frame1, frame2

# Использование
#syncer = VideoSynchronizer("video1.mp4", "video2.mp4")
#for frame1, frame2 in syncer.get_sync_frames():
 #   cv2.imshow('Sync', np.hstack((frame1, frame2)))
  #  if cv2.waitKey(25) & 0xFF == ord('q'):
   #     break


class StereoCalibrator:
    def __init__(self, chessboard_size=(9,6), square_size=2.5):
        self.chessboard_size = chessboard_size
        self.square_size = square_size
        self.obj_points = []
        self.img_points_left = []
        self.img_points_right = []
        
        # Подготовка 3D точек шахматки
        self.objp = np.zeros((chessboard_size[0]*chessboard_size[1],3), np.float32)
        self.objp[:,:2] = np.mgrid[0:chessboard_size[0],0:chessboard_size[1]].T.reshape(-1,2) * square_size

    def process_videos(self, video1, video2):
        syncer = VideoSynchronizer(video1, video2)
        criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)
        
        for frame1, frame2 in syncer.get_sync_frames():
            gray1 = cv2.cvtColor(frame1, cv2.COLOR_BGR2GRAY)
            gray2 = cv2.cvtColor(frame2, cv2.COLOR_BGR2GRAY)
            
            # Поиск углов
            ret1, corners1 = cv2.findChessboardCorners(gray1, self.chessboard_size, None)
            ret2, corners2 = cv2.findChessboardCorners(gray2, self.chessboard_size, None)
            
            if ret1 and ret2:
                corners1 = cv2.cornerSubPix(gray1, corners1, (11,11), (-1,-1), criteria)
                corners2 = cv2.cornerSubPix(gray2, corners2, (11,11), (-1,-1), criteria)
                
                self.obj_points.append(self.objp)
                self.img_points_left.append(corners1)
                self.img_points_right.append(corners2)
                
        # Индивидуальная калибровка
        ret, mtx1, dist1, _, _ = cv2.calibrateCamera(
            self.obj_points, self.img_points_left, gray1.shape[::-1], None, None)
        
        ret, mtx2, dist2, _, _ = cv2.calibrateCamera(
            self.obj_points, self.img_points_right, gray2.shape[::-1], None, None)
        
        # Стереокалибровка
        ret, mtx1, dist1, mtx2, dist2, R, T, E, F = cv2.stereoCalibrate(
            self.obj_points, self.img_points_left, self.img_points_right,
            mtx1, dist1, mtx2, dist2, gray1.shape[::-1],
            criteria=(cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 100, 1e-5),
            flags=cv2.CALIB_FIX_INTRINSIC
        )
        
        return {
            'mtx1': mtx1, 'dist1': dist1,
            'mtx2': mtx2, 'dist2': dist2,
            'R': R, 'T': T,
            'E': E, 'F': F
        }

# Пример использования
calibrator = StereoCalibrator()
print("start")
params = calibrator.process_videos("left_cam.mp4", "right_cam.mp4")
np.savez("stereo_params.npz", **params)
print("finish")

start


In [ ]:
import cv2
import numpy as np
import glob

class CameraCalibrator:
    def __init__(self, pattern_size=(7,7), square_size=0.025):
        self.pattern_size = pattern_size
        self.square_size = square_size
        self.obj_points = []
        self.img_points = []
        
        self.objp = np.zeros((pattern_size[0]*pattern_size[1],3), np.float32)
        self.objp[:,:2] = np.mgrid[0:pattern_size[0],0:pattern_size[1]].T.reshape(-1,2) * square_size

    def calibrate(self, images_folder):
        images = glob.glob(f"{images_folder}/*.jpg") + glob.glob(f"{images_folder}/*.png")
        
        if not images:
            raise FileNotFoundError(f"Нет изображений в папке {images_folder}!")
            
        criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)
        found = 0
        
        for fname in images:
            img = cv2.imread(fname)
            if img is None:
                continue
                
            gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            ret, corners = cv2.findChessboardCorners(gray, self.pattern_size, None)
            
            if ret:
                corners = cv2.cornerSubPix(gray, corners, (11,11), (-1,-1), criteria)
                self.obj_points.append(self.objp)
                self.img_points.append(corners)
                found += 1
                print(f"Найдено углов в {fname}")

        if found == 0:
            raise ValueError("Шахматная доска не найдена ни на одном изображении!")
            
        h, w = img.shape[:2]
        ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(
            self.obj_points, self.img_points, (w, h), None, None)
            
        print(f"\nУспешно обработано изображений: {found}/{len(images)}")
        return mtx, dist

# Пример использования
try:
    calibrator = CameraCalibrator(pattern_size=(7,7), square_size=0.025)
    mtx, dist = calibrator.calibrate("frames_all_left_cam_mihon")
    np.savez("left_cam_mihon_colib.npz", camera_matrix=mtx, dist_coeffs=dist)

    calibrator2 = CameraCalibrator(pattern_size=(7,7), square_size=0.025)
    mtx, dist = calibrator2.calibrate("frames_all_right_cam_danya")
    np.savez("right_cam_danya_colib.npz", camera_matrix=mtx, dist_coeffs=dist)
except Exception as e:
    print(f"Ошибка: {e}")


Найдено углов в frames_all_left_cam_mihon\frame_0000.jpg
Найдено углов в frames_all_left_cam_mihon\frame_0001.jpg
Найдено углов в frames_all_left_cam_mihon\frame_0002.jpg
Найдено углов в frames_all_left_cam_mihon\frame_0003.jpg
Найдено углов в frames_all_left_cam_mihon\frame_0004.jpg
Найдено углов в frames_all_left_cam_mihon\frame_0005.jpg
Найдено углов в frames_all_left_cam_mihon\frame_0006.jpg
Найдено углов в frames_all_left_cam_mihon\frame_0007.jpg
Найдено углов в frames_all_left_cam_mihon\frame_0008.jpg
Найдено углов в frames_all_left_cam_mihon\frame_0009.jpg
Найдено углов в frames_all_left_cam_mihon\frame_0010.jpg
Найдено углов в frames_all_left_cam_mihon\frame_0011.jpg
Найдено углов в frames_all_left_cam_mihon\frame_0012.jpg
Найдено углов в frames_all_left_cam_mihon\frame_0013.jpg
Найдено углов в frames_all_left_cam_mihon\frame_0014.jpg
Найдено углов в frames_all_left_cam_mihon\frame_0015.jpg
Найдено углов в frames_all_left_cam_mihon\frame_0016.jpg
Найдено углов в frames_all_left

In [2]:
import numpy as np
import cv2
import os

def print_calibration_metrics(calib_file):
    # Проверка существования файла
    if not os.path.exists(calib_file):
        print(f"Файл {calib_file} не найден!")
        print("Доступные файлы в текущей директории:")
        print(os.listdir())
        return
    
    data = np.load(calib_file)
    
    print("\n=== Camera Matrix ===")
    print(data['camera_matrix'])
    
    print("\n=== Distortion Coefficients ===")
    print(data['dist_coeffs'])
    
    # Расчет фокусного расстояния
    fx = data['camera_matrix'][0,0]
    fy = data['camera_matrix'][1,1]
    print(f"\nFocal Length (fx): {fx:.1f} px")
    print(f"Focal Length (fy): {fy:.1f} px")
    
    # Оптический центр
    cx = data['camera_matrix'][0,2]
    cy = data['camera_matrix'][1,2]
    print(f"Optical Center: ({cx:.1f}, {cy:.1f})")

# Проверяем существование файлов перед вызовом
print("Проверка файлов:")
print("left_cam_mihon_colib.npz exists:", os.path.exists("left_cam_mihon_colib.npz"))
print("right_cam_calib.npz exists:", os.path.exists("right_cam_calib.npz"))

# Пример использования
print("\nleft_cam info:")
print_calibration_metrics("left_cam_mihon_colib.npz")  # Исправлено имя файла

print("\nright_cam info:")
print_calibration_metrics("right_cam_calib.npz")


Проверка файлов:
left_cam_mihon_colib.npz exists: True
right_cam_calib.npz exists: True

left_cam info:

=== Camera Matrix ===
[[733.69078545   0.         232.164899  ]
 [  0.         748.83031112 423.89795623]
 [  0.           0.           1.        ]]

=== Distortion Coefficients ===
[[ 1.12119763e-01  6.22426262e-01  3.74884748e-04 -1.85916773e-03
  -3.03870070e+00]]

Focal Length (fx): 733.7 px
Focal Length (fy): 748.8 px
Optical Center: (232.2, 423.9)

right_cam info:

=== Camera Matrix ===
[[983.58100649   0.         478.97924058]
 [  0.         983.88686381 644.65400484]
 [  0.           0.           1.        ]]

=== Distortion Coefficients ===
[[ 1.97392074e-01 -1.30238448e+00  1.12036419e-03  6.39925513e-04
   2.63344168e+00]]

Focal Length (fx): 983.6 px
Focal Length (fy): 983.9 px
Optical Center: (479.0, 644.7)


In [ ]:
import cv2
import numpy as np
from sort import Sort
from ultralytics import YOLO
import torch

class StereoDistanceCalculator:
    def __init__(self, left_calib, right_calib, baseline=0.25):
        self.left_params = np.load(left_calib)
        self.right_params = np.load(right_calib)
        
        # Инициализация стереоалгоритма с оптимальными параметрами
        self.stereo = cv2.StereoSGBM_create(
            minDisparity=0,
            numDisparities=128,    # Должно быть кратно 16
            blockSize=5,
            P1=8*3*5**2,
            P2=32*3*5**2,
            disp12MaxDiff=5,
            uniquenessRatio=15,
            speckleWindowSize=50,
            speckleRange=2,
            mode=cv2.STEREO_SGBM_MODE_SGBM_3WAY
        )
        
        self.baseline = baseline
        self.focal_length = np.mean([
            self.left_params['camera_matrix'][0,0],
            self.right_params['camera_matrix'][0,0]
        ])

    def _preprocess_images(self, left, right):
        # Приведение к одинаковому размеру
        h, w = left.shape[:2]
        right = cv2.resize(right, (w, h))
        
        # Конвертация в grayscale
        left_gray = cv2.cvtColor(left, cv2.COLOR_BGR2GRAY)
        right_gray = cv2.cvtColor(right, cv2.COLOR_BGR2GRAY)
        
        # Удаление дисторсии
        left_rect = cv2.undistort(left_gray, 
                                self.left_params['camera_matrix'],
                                self.left_params['dist_coeffs'])
        right_rect = cv2.undistort(right_gray,
                                 self.right_params['camera_matrix'],
                                 self.right_params['dist_coeffs'])
        
        return left_rect, right_rect

    def calculate_distance(self, left_frame, right_frame, bbox):
        try:
            # Проверка входных данных
            if left_frame is None or right_frame is None:
                raise ValueError("Пустой кадр")
                
            if left_frame.shape != right_frame.shape:
                left_frame = cv2.resize(left_frame, (right_frame.shape[1], right_frame.shape[0]))
                
            # Предобработка изображений
            left_rect, right_rect = self._preprocess_images(left_frame, right_frame)

            cv2.imshow('1', left_rect)
            cv2.imshow('2', right_rect)  #проверка что в кадре
            cv2.waitKey(0)
            cv2.destroyAllWindows()

            
            # Вычисление диспарантности
            disparity = self.stereo.compute(left_rect, right_rect)
            
            # Фильтрация шумов
            disparity = cv2.medianBlur(disparity, 5)
            disparity = np.abs(disparity)
            
            # Расчет расстояния для ROI
            x1, y1, x2, y2 = map(int, bbox)
            roi = disparity[y1:y2, x1:x2]
            valid_disp = roi[roi > 0]
            
            if valid_disp.size == 0:
                return 0.0
                
            avg_disp = np.mean(valid_disp)
            Z = (self.focal_length * self.baseline) / (avg_disp + 1e-6)
            return Z
            
        except Exception as e:
            print(f"Ошибка расчета расстояния: {str(e)}")
            return 0.0

class ObjectDetection:
    def __init__(self, video_left, video_right, output_file="output_.avi"):
        self.device = 'cuda' if torch.cuda.is_available() else 'cpu'
        print("Using Device: ", self.device)
        self.cap_left = cv2.VideoCapture(video_left)
        self.cap_right = cv2.VideoCapture(video_right)
        self.model = YOLO("yolov8m.pt")
        self.tracker = Sort(max_age=50, min_hits=8, iou_threshold=0.50)
        
        # Инициализация модуля расчета расстояний
        self.distance_calculator = StereoDistanceCalculator(
            "left_cam_diman_calib.npz",
            "right_cam_mihon_colib.npz",
            baseline=0.19  # Ваше измеренное расстояние между камерами
        )
        
        self.trajectories = {}
        self.speed_history = {}

    def process_frame(self):
        ret_left, left_frame = self.cap_left.read()
        ret_right, right_frame = self.cap_right.read()
        if not ret_left or not ret_right: return None
        
        results = self.model(left_frame)
        boxes = results[0].boxes.xyxy.cpu().numpy()
        confidences = results[0].boxes.conf.cpu().numpy()
        classes = results[0].boxes.cls.cpu().numpy()
        
        # Фильтрация объектов
        detected_objects = []
        for box, conf, cls in zip(boxes, confidences, classes):
            detected_objects.append([*box, conf])
        
        tracks = self.tracker.update(np.array(detected_objects))
        
        # Расчет расстояний и отрисовка
        for track in tracks:
            x1, y1, x2, y2, track_id = map(int, track)
            bbox = [x1, y1, x2, y2]
            
            try:
                distance = self.distance_calculator.calculate_distance(
                    left_frame, right_frame, bbox)
                
                # Отрисовка рамки и информации
                cv2.rectangle(left_frame, (x1,y1), (x2,y2), (0,255,0), 2)
                cv2.putText(left_frame, f"ID:{track_id} Dist:{distance:.2f}m",
                          (x1, y1-10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255,0,0), 2)
            except Exception as e:
                print(f"Ошибка расчета расстояния: {e}")
        
        return left_frame

    def run(self):
        while True:
            frame = self.process_frame()
            if frame is None: break
            
            cv2.imshow('Stereo Detection', frame)
            if cv2.waitKey(1) == ord('q'): break
        
        self.cap_left.release()
        self.cap_right.release()
        cv2.destroyAllWindows()

# Запуск системы
detector = ObjectDetection("http://192.168.0.100:4747/video", "http://192.168.0.103:4747/video")
detector.run()


Using Device:  cuda

0: 480x640 1 cup, 1 refrigerator, 58.7ms
Speed: 0.0ms preprocess, 58.7ms inference, 129.6ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 cup, 1 refrigerator, 237.3ms
Speed: 2.0ms preprocess, 237.3ms inference, 15.8ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 refrigerator, 175.0ms
Speed: 1.5ms preprocess, 175.0ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 cup, 1 refrigerator, 26.1ms
Speed: 0.0ms preprocess, 26.1ms inference, 2.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 cup, 1 refrigerator, 213.3ms
Speed: 0.0ms preprocess, 213.3ms inference, 15.9ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 cup, 1 refrigerator, 190.7ms
Speed: 0.0ms preprocess, 190.7ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 cup, 1 refrigerator, 174.8ms
Speed: 4.1ms preprocess, 174.8ms inference, 15.7ms postprocess per image at shape (1, 3, 480, 640)